# Aprendizaje Multietiqueta de Patrones Geométricos en Objetos de Herencia Cultural
# Kunisch Features from ResNet architectures
## Seminario de Tesis II, Primavera 2022
### Master of Data Science. Universidad de Chile.
#### Prof. guía: Benjamín Bustos - Prof. coguía: Iván Sipirán
#### Autor: Matías Vergara


## Imports

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy
import pandas as pd
import math
import random
import shutil

from torch.utils.data import Dataset
from PIL import Image

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np, scipy.io
import argparse
import json

## Mounting Google Drive

In [9]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_path = 'drive/MyDrive/TesisMV/'
except:
    folder_path = '../'

In [10]:
os.listdir(folder_path)

['.git',
 '.gitignore',
 '.idea',
 'actas',
 'entregables',
 'features',
 'graficos',
 'labels',
 'models',
 'notebooks',
 'outputs',
 'patterns',
 'presentaciones',
 'README.md',
 'requirements.txt',
 'tables']

## Dataset and model selection

In [11]:
#modify only this cell
USE_RN50 = True
SUBCHAPTERS = False
DS_FLAGS = []
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
CROP_TIMES = 1
RANDOM_TIMES = 2
ELASTIC_TIMES = 1

In [13]:
# This cells builds the data_flags variable, that will be used
# to map the requested data treatment to folders
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = ['crop', 'randaug', 'elastic']
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

subchapter_str = 'subchapters/' if SUBCHAPTERS else ''
patterns_path = folder_path + 'patterns/' + subchapter_str + data_flags + "/"
labels_path = folder_path + 'labels/' +  subchapter_str + data_flags + "/"
# models_path = folder_path + 'models/' + subchapter_str + (f'resnet50_{data_flags}.pth' if USE_RN50 else f'resnet18_{data_flags}.pth')
# features_path = folder_path + 'features/' + subchapter_str + (f'resnet50_{data_flags}/' if USE_RN50 else f'resnet18_{data_flags}/')

#rn = 50
#ep = 65
#models_path = folder_path + f"models/resnet{rn}_blur_each5/resnet{rn}_blur_e{ep}.pth"
#features_path = folder_path + f"features/resnet{rn}_blur_each5/resnet{rn}_blur_e{ep}/"
rn = 50 if USE_RN50 else 18
models_path = folder_path + f"models/resnet/resnet{rn}_{data_flags}.pth"
features_path = folder_path + f"features/resnet/resnet{rn}_{data_flags}/"


if not (os.path.isdir(patterns_path) and os.path.isdir(labels_path)):
    raise FileNotFoundError("No existen directorios de datos para el conjunto de flags seleccionado. Verifique que el dataset exista y, de lo contrario, llame a Split and Augmentation")

if not (os.path.isfile(models_path)):
    raise FileNotFoundError(f"No se encontró modelo para el conjunto de flags seleccionado. Verifique que el modelo exista y, de lo contrario, llame a ResNet Retraining\n{models_path}")


print("Pattern set encontrado en {}".format(patterns_path))
print("Labels set encontrado en {}".format(labels_path))
print("Modelo encontrado en {}".format(models_path))

Pattern set encontrado en ../patterns/base/
Labels set encontrado en ../labels/base/
Modelo encontrado en ../models/resnet/resnet50_base.pth


## Dataset loader

In [14]:
class PatternDataset(Dataset):
    def __init__(self, root_dir, transform=None, build_classification=False, name_cla='output.cla'):
        self.root_dir = root_dir
        self.transform = transform
        self.namefiles = []


        self.classes = sorted(os.listdir(self.root_dir))

        for cl in self.classes:
            for pat in os.listdir(os.path.join(self.root_dir, cl)):
                self.namefiles.append((pat, cl))

        print(f'Files:{len(self.namefiles)}')
        self.namefiles = sorted(self.namefiles, key = lambda x: x[0])

        if build_classification:
            dictClasses = dict()

            for cl in self.classes:
                dictClasses[cl] = []

            for index, (name, cl) in enumerate(self.namefiles):
                dictClasses[cl].append((name, index))

            with open(name_cla, 'w') as f:
                f.write('PSB 1\n')
                f.write(f'{len(self.classes)} {len(self.namefiles)}\n')
                f.write('\n')
                for cl in self.classes:
                    f.write(f'{cl} 0 {len(dictClasses[cl])}\n')
                    for item in dictClasses[cl]:
                        f.write(f'{item[1]}\n')
                    f.write('\n')

    def __len__(self):
        return len(self.namefiles)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        img_name = os.path.join(self.root_dir, self.namefiles[index][1], self.namefiles[index][0])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return self.namefiles[index], image


## Preparation

In [15]:
random.seed(30)

output_train = features_path + "augmented_train_df.json"
output_val = features_path + "val_df.json"
output_test = features_path + "test_df.json"

train_df = pd.read_json(labels_path + "augmented_train_df.json", orient='index')
val_df = pd.read_json(labels_path + "val_df.json", orient='index')
test_df = pd.read_json(labels_path + "test_df.json", orient='index')

train_pts = train_df.index.values
val_pts = val_df.index.values
test_pts = test_df.index.values

## Extraction

In [16]:
def imshow(inp, title = None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)

    plt.imshow(inp)
    plt.show()

def get_vector(model,layer, dim_embedding, x):

  my_embedding = torch.zeros(dim_embedding)

  def copy_data(m,i,o):
    my_embedding.copy_(o.data.squeeze())

  h = layer.register_forward_hook(copy_data)
  model(x)
  h.remove()

  return my_embedding

device = ('cuda' if torch.cuda.is_available() else 'cpu')

my_transform = transforms.Compose([ transforms.Resize(224),
                                    #transforms.CenterCrop(224),
                     transforms.ToTensor(),
                     transforms.Normalize(mean=[0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
                    ])

dataTrain = PatternDataset(root_dir=os.path.join(patterns_path, 'train'), transform=my_transform)
dataVal = PatternDataset(root_dir=os.path.join(patterns_path, 'val'), transform=my_transform)
dataTest = PatternDataset(root_dir=os.path.join(patterns_path, 'test'), transform=my_transform)

loaderTrain = DataLoader(dataTrain)
loaderVal = DataLoader(dataVal)
loaderTest = DataLoader(dataTest)

if USE_RN50:
    model = models.resnet50(pretrained = True)
else:
    model = models.resnet18(pretrained = True)
dim = model.fc.in_features

output_dim = 20 if SUBCHAPTERS else 6
model.fc = nn.Linear(dim, output_dim)

model = model.to(device)

try:
    model.load_state_dict(torch.load(models_path))
except RuntimeError as e:
    print('Ignoring "' + str(e) + '"')

layer = model._modules.get('avgpool')

model.eval()

#features = []
features_train = {}
features_val = {}
features_test = {}


for name, img in loaderTrain:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_train[code] = feat.numpy().tolist()
  #features.append(feat.numpy())

for name, img in loaderVal:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_val[code] = feat.numpy().tolist()
  #features.append(feat.numpy())

for name, img in loaderTest:
  feat = get_vector(model, layer, dim, img.to(device))
  namefile = name[0][0]
  code, rest = namefile.split('.')
  features_test[code] = feat.numpy().tolist()
#features = np.vstack(features)
#print(features.shape)

Files:542
Files:78
Files:156


In [17]:
os.makedirs(features_path, exist_ok=True)

In [18]:
features_train_df = pd.DataFrame.from_dict(features_train, orient='index')
features_val_df = pd.DataFrame.from_dict(features_val, orient='index')
features_test_df = pd.DataFrame.from_dict(features_test, orient='index')

features_train_df.to_json(output_train, orient='index')
features_val_df.to_json(output_val, orient='index')
features_test_df.to_json(output_test, orient='index')

In [19]:
features_train_df

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
10a,0.220067,0.192842,0.230825,0.030218,0.096193,0.003017,0.056004,0.008634,0.103588,0.044708,...,0.239739,0.009333,0.134025,0.021282,0.064552,0.085574,0.073898,0.001953,0.121407,0.000000
10c,0.299684,0.205679,0.209356,0.100046,0.231695,0.010343,0.085307,0.059835,0.080415,0.020939,...,0.172179,0.069838,0.180563,0.033630,0.004924,0.061860,0.019382,0.033054,0.126834,0.028692
10d,0.270987,0.266924,0.061249,0.060208,0.235613,0.061621,0.070450,0.170796,0.071693,0.012005,...,0.058542,0.167929,0.165272,0.025749,0.001131,0.102660,0.013620,0.010305,0.108796,0.001071
10e,0.199509,0.294608,0.002447,0.088599,0.225461,0.131101,0.032750,0.138863,0.006781,0.000518,...,0.001919,0.100851,0.242753,0.028078,0.000662,0.063491,0.018281,0.015822,0.253149,0.000000
11a,0.273982,0.282333,0.032810,0.020401,0.038270,0.494274,0.029916,0.213418,0.025561,0.002748,...,0.006390,0.144122,0.009766,0.001530,0.000000,0.018667,0.094121,0.001674,0.340465,0.000318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96i,0.071190,0.088913,0.030630,0.017437,0.175513,0.006621,0.158218,0.000796,0.337129,0.101432,...,0.058232,0.006009,0.226061,0.017694,0.014825,0.054114,0.011399,0.000019,0.048853,0.000000
9a,0.500378,0.172919,0.978929,0.177229,0.197529,0.022774,0.131688,0.112513,0.095020,0.043275,...,0.766178,0.078866,0.262157,0.010490,0.012799,0.013864,0.030137,0.045483,0.527473,0.030071
9b,0.311907,0.122832,0.639037,0.084956,0.116858,0.001267,0.110946,0.032009,0.076971,0.010310,...,0.493930,0.049394,0.257918,0.004122,0.009260,0.007448,0.021136,0.009069,0.360306,0.008532
9c,0.380951,0.144732,0.787097,0.082002,0.104264,0.000000,0.095917,0.067211,0.057363,0.006513,...,0.535690,0.075754,0.165313,0.008245,0.033935,0.026949,0.063833,0.011848,0.395545,0.011779
